In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySparkPostgreSQL") \
    .config("spark.jars", "/Users/jayakrish/jdbc/postgresql-42.7.5.jar") \
    .getOrCreate()


25/02/25 15:09:00 WARN Utils: Your hostname, jayakrishs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.119 instead (on interface en0)
25/02/25 15:09:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/02/25 15:09:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
db_properties = {
    "user": "postgres",
    "password": "pgroot",
    "driver": "org.postgresql.Driver"
}

customers_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/customer_db") \
    .option("dbtable", "customers") \
    .options(**db_properties) \
    .load()

transactions_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/customer_db") \
    .option("dbtable", "transactions") \
    .options(**db_properties) \
    .load()

customers_df.show()
transactions_df.show()


+-----------+-------------+-----------------+-------+
|customer_id|         name|            email|country|
+-----------+-------------+-----------------+-------+
|          1|Alice Johnson|alice@example.com|    USA|
|          2|    Bob Smith|  bob@example.com| Canada|
+-----------+-------------+-----------------+-------+

+--------------+-----------+------+-------------------+
|transaction_id|customer_id|amount|   transaction_date|
+--------------+-----------+------+-------------------+
|             1|          1|200.50|2024-02-20 12:30:00|
|             2|          2|450.00|2024-02-21 15:45:00|
+--------------+-----------+------+-------------------+



In [3]:
joined_df = customers_df.join(transactions_df, "customer_id", "inner")
joined_df.show()


+-----------+-------------+-----------------+-------+--------------+------+-------------------+
|customer_id|         name|            email|country|transaction_id|amount|   transaction_date|
+-----------+-------------+-----------------+-------+--------------+------+-------------------+
|          1|Alice Johnson|alice@example.com|    USA|             1|200.50|2024-02-20 12:30:00|
|          2|    Bob Smith|  bob@example.com| Canada|             2|450.00|2024-02-21 15:45:00|
+-----------+-------------+-----------------+-------+--------------+------+-------------------+



In [4]:
from pyspark.sql.functions import sum

spending_df = joined_df.groupBy("customer_id").agg(sum("amount").alias("total_spent"))
spending_df.show()


+-----------+-----------+
|customer_id|total_spent|
+-----------+-----------+
|          1|     200.50|
|          2|     450.00|
+-----------+-----------+



In [5]:
spending_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/customer_db") \
    .option("dbtable", "customer_spending") \
    .options(**db_properties) \
    .mode("overwrite") \
    .save()
